In [28]:
import pandas as pd
from sklearn import datasets, model_selection
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

<h3>Load Datasets</h3>

In [29]:
cancer = datasets.load_breast_cancer()
dtcancer = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
dttennis = pd.read_csv("../data/play_tennis.csv")

<h3>Split Datasets</h3>

In [30]:
ctrain, ctest = model_selection.train_test_split(dtcancer, test_size=0.2, train_size=0.8, random_state=1)
tntrain, tntest = model_selection.train_test_split(dttennis, test_size=0.2, train_size=0.8, random_state=1)

<h3>Encode Play Tennis Column</h3>

In [31]:
label_encoder = LabelEncoder()
for cl in tntrain:
    tntrain[cl] = label_encoder.fit_transform(tntrain[cl])
    tntest[cl] = label_encoder.fit_transform(tntest[cl])

C:\Users\BENIDI~1\AppData\Local\Temp/ipykernel_7220/1993091258.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tntrain[cl] = label_encoder.fit_transform(tntrain[cl])
C:\Users\BENIDI~1\AppData\Local\Temp/ipykernel_7220/1993091258.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tntest[cl] = label_encoder.fit_transform(tntest[cl])


<h3>Decision Tree Classifier</h3>

<h3>Id3 Estimator</h3>

<h3>KMeans</h3>

In [36]:
ckmeans = KMeans(n_clusters=2, random_state=42).fit(ctrain)
tnkmeans = KMeans(n_clusters=2, random_state=42).fit(tntrain)

ckmeanspredict = ckmeans.predict(ctest)
tnkmeanspredict = tnkmeans.predict(tntest)


array([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0])

<h3>Logistic Regression</h3>

<h3>Neural Network</h3>

<h3>SVM</h3>